# Regex annotation

In [17]:
import numpy       as np
import os
import json

data_directory = "/enc_data/eddata/organized-data/"

## Regex + parsing

In [2]:
with open('pacemaker_words.json', 'r') as f: 
    pacemaker_words = json.load(f)

In [8]:
positive_filenames = set([])
negative_filenames = set([])

for patient_id in os.listdir(data_directory): 
    is_pacemaker = False
    
    for report_filename in os.listdir(data_directory + patient_id): 
        if not report_filename.endswith('.res') or is_pacemaker: 
            continue
        
        with open(data_directory + patient_id + '/' + report_filename, 'r') as f: 
            for row in f: 
                splitted = set(row.lower().replace('/', ' / ').split())
                
                if any([word in splitted for word in pacemaker_words]): 
                    is_pacemaker = True
                    positive_filenames.add(patient_id + "/" + report_filename)
                    break
                    
    if not is_pacemaker: 
        negative_filenames.add(patient_id)
        
with open("results/regex_ann/pos.json", 'w') as f: 
    json.dump(list(positive_filenames), f)
with open("results/regex_ann/neg.json", 'w') as f: 
    json.dump(list(negative_filenames), f)

In [9]:
print len(positive_filenames), len(negative_filenames)

2147 43875


## Evaluation

In [18]:
true_pos = set([])
predicted_pos = set([])

with open('/enc_data/eddata/clipnum_with_pacemaker.txt', 'r') as f: 
    for row in f: 
        pos = row.split('/')[0]
        if os.path.exists(data_directory + pos): 
            true_pos.add(pos)
        
with open('results/regex_ann/pos.json', 'r') as f: 
    predicted_filenames = json.load(f)
    for filename in predicted_filenames: 
        predicted_pos.add(filename.split("/")[0])
        
print np.all([ pos in predicted_pos for pos in true_pos ])

True


Sensitivity: 100%
PPV: 100%